In [151]:
%reload_ext autoreload
%autoreload 2

from src.file_utils import download_file, decompress_gz,make_dir
import glob
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
from tqdm import tqdm
from src.parse_warc import extract_japanese_from_warc

import json
import gzip

In [153]:
json_path_list=glob.glob('data/gz/*.gz')
len(json_path_list)

528

In [14]:
path="data/jap_dump/crawl-data_CC-MAIN-2023-50_segments_1700679099281.67_warc_CC-MAIN-20231128083443-20231128113443-00000.warc_jap.gz.gz"
# gzipで圧縮されたJSONファイルを読み込み
with gzip.open(path, 'rt', encoding='utf-8') as f:
    data_loaded = json.load(f)


In [59]:
"""
from bunkai import Bunkai
bunkai = Bunkai()
def parse_lines_bunkai(lines):
    lines = lines.strip().replace("\n", "\u2581")
    return bunkai("".join(lines))

"""

'\nfrom bunkai import Bunkai\nbunkai = Bunkai()\ndef parse_lines_bunkai(lines):\n    lines = lines.strip().replace("\n", "▁")\n    return bunkai("".join(lines))\n\n'

In [133]:
from src.cleaner.auto_filter import do_filter
from src.cleaner import text_normalizer
from src.cleaner.junk_filter import remove_junk_lines
lines=data_loaded[23]["text"]
lines=[i[0] for i in lines]
#lines=remove_junk_lines(lines)
text="\n".join(lines)

In [140]:
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(て)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)
lines=list(segmenter(lines))


In [150]:
lines

['URLからファイルシステム上の位置へのマップ-Apache HTTPサーババージョン2.4モジュール|ディレクティブ|FAQ|用語|サイトマップApache HTTPサーババージョン2.4Apache>HTTPサーバ>ドキュメンテーション>バージョン2.4URLからファイルシステム上の位置へのマップ翻訳済み言語:en|fr|ja|ko|trこの日本語訳はすでに古くなっている可能性があります。',
 '最近更新された内容を見るには英語版をご覧下さい。',
 'この文書はApacheがリクエストのURLから送信するファイルのファイルシステム上の位置を決定する方法を説明します。',
 '関連するモジュールとディレクティブDocumentRootDocumentRoot外のファイルユーザディレクトリURLリダイレクションリバースプロキシリライトエンジンFile Not Found参照コメント関連するモジュールとディレクティブ関連モジュール関連ディレクティブmod_aliasmod_proxymod_rewritemod_userdirmod_spelingmod_vhost_aliasAliasAliasMatchCheckSpellingDocumentRootErrorDocumentOptionsProxyPassProxyPassReverseProxyPassReverseCookieDomainProxyPassReverseCookiePathRedirectRedirectMatchRewriteCondRewriteMatchScriptAliasScriptAliasMatchUserDirDocumentRootリクエストに対してどのファイルを送信するかを決定するときのApacheのデフォルトの動作は、リクエストのURL-Path(URLのホスト名とポート番号の後に続く部分)を取り出して設定ファイルで指定されているDocumentRootの最後に追加する、というものです。',
 'ですから、DocumentRootの下のディレクトリやファイルがウェブから見える基本のドキュメントの木構造をなします。',
 'Apacheにはサーバが複数のホストへのリクエストを受け取るバーチャルホストの機能もあります。',
 'この場合、それぞれのバーチャルホストに対

In [148]:



# 文末記号のリスト（全角と半角）
sentence_endings = ['。', '！', '？', '.', '!', '?',"．"]
touten_symbols=["､",",","，","、"]

def is_sentence_ending(text):
    """指定された文字が文末記号かどうかを判断する"""
    return text[-1] in sentence_endings

# 結果を格納するリスト

grouped_lines=[]

temp_text=""
old_line=""
for line in lines:
    line=text_normalizer.normalize(line) #文字の正規化

    #重複行は削除
    if line==old_line:
        old_line=line
        continue

    old_line=line

    #if old_line[-1] in touten_symbols:
    #    temp_text+=line
    #    #print(temp_text)
    #    continue

    # ｡ などで終わらないlineの場合は､改行する
    if not is_sentence_ending(line):
        if temp_text!="":
            grouped_lines.append(temp_text)
            #print(temp_text)
        temp_text=""
        grouped_lines.append(line)
        #print(line)

    else:
        temp_text+=line
    #if not is_sentence_ending(temp_text):
    #    grouped_lines.append(temp_text)
    #    temp_text=""

text="\n".join(grouped_lines)
#text=do_filter(text)
print(text)


#TODO: まず、  をしてください(無料)。 のようになってしまう｡ 「､」で続くlineは､改行せずに統合すれば解決?

URLからファイルシステム上の位置へのマップ-Apache HTTPサーババージョン2.4モジュール|ディレクティブ|FAQ|用語|サイトマップApache HTTPサーババージョン2.4Apache>HTTPサーバ>ドキュメンテーション>バージョン2.4URLからファイルシステム上の位置へのマップ翻訳済み言語:en|fr|ja|ko|trこの日本語訳はすでに古くなっている可能性があります。最近更新された内容を見るには英語版をご覧下さい。この文書はApacheがリクエストのURLから送信するファイルのファイルシステム上の位置を決定する方法を説明します。関連するモジュールとディレクティブDocumentRootDocumentRoot外のファイルユーザディレクトリURLリダイレクションリバースプロキシリライトエンジンFile Not Found参照コメント関連するモジュールとディレクティブ関連モジュール関連ディレクティブmod_aliasmod_proxymod_rewritemod_userdirmod_spelingmod_vhost_aliasAliasAliasMatchCheckSpellingDocumentRootErrorDocumentOptionsProxyPassProxyPassReverseProxyPassReverseCookieDomainProxyPassReverseCookiePathRedirectRedirectMatchRewriteCondRewriteMatchScriptAliasScriptAliasMatchUserDirDocumentRootリクエストに対してどのファイルを送信するかを決定するときのApacheのデフォルトの動作は、リクエストのURL-Path(URLのホスト名とポート番号の後に続く部分)を取り出して設定ファイルで指定されているDocumentRootの最後に追加する、というものです。ですから、DocumentRootの下のディレクトリやファイルがウェブから見える基本のドキュメントの木構造をなします。Apacheにはサーバが複数のホストへのリクエストを受け取るバーチャルホストの機能もあります。この場合、それぞれのバーチャルホストに対して違うDocumentRootを指定することができます。また、

In [149]:
grouped_lines

['URLからファイルシステム上の位置へのマップ-Apache HTTPサーババージョン2.4モジュール|ディレクティブ|FAQ|用語|サイトマップApache HTTPサーババージョン2.4Apache>HTTPサーバ>ドキュメンテーション>バージョン2.4URLからファイルシステム上の位置へのマップ翻訳済み言語:en|fr|ja|ko|trこの日本語訳はすでに古くなっている可能性があります。最近更新された内容を見るには英語版をご覧下さい。この文書はApacheがリクエストのURLから送信するファイルのファイルシステム上の位置を決定する方法を説明します。関連するモジュールとディレクティブDocumentRootDocumentRoot外のファイルユーザディレクトリURLリダイレクションリバースプロキシリライトエンジンFile Not Found参照コメント関連するモジュールとディレクティブ関連モジュール関連ディレクティブmod_aliasmod_proxymod_rewritemod_userdirmod_spelingmod_vhost_aliasAliasAliasMatchCheckSpellingDocumentRootErrorDocumentOptionsProxyPassProxyPassReverseProxyPassReverseCookieDomainProxyPassReverseCookiePathRedirectRedirectMatchRewriteCondRewriteMatchScriptAliasScriptAliasMatchUserDirDocumentRootリクエストに対してどのファイルを送信するかを決定するときのApacheのデフォルトの動作は、リクエストのURL-Path(URLのホスト名とポート番号の後に続く部分)を取り出して設定ファイルで指定されているDocumentRootの最後に追加する、というものです。ですから、DocumentRootの下のディレクトリやファイルがウェブから見える基本のドキュメントの木構造をなします。Apacheにはサーバが複数のホストへのリクエストを受け取るバーチャルホストの機能もあります。この場合、それぞれのバーチャルホストに対して違うDocumentRootを指定することができます。ま

In [139]:
line[-1]

'、'

In [146]:
line

'.Apache License,Version 2.0モジュール|ディレクティブ|FAQ|用語|サイトマップモジュールディレクティブFAQ用語サイトマップ'